In [1]:
#pip install ta
#pip install Backtesting 
#pip install Binance

#Kütüphaneleri eklenmeli 

In [2]:
import pandas as pd
import ta
from backtesting import Backtest, Strategy 
from backtesting.lib import crossover 
from binance import Client 
client = Client()


C:\Users\Dell\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
def getdata(symbol, start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'4h',start))
    
    frame = frame[[0,1,2,3,4]]
    frame.columns = ['Date','Open','High','Low','Close']
    frame.Date = pd.to_datetime(frame.Date, unit='ms')
    frame.set_index('Date',inplace=True)
    frame = frame.astype(float)
    return frame

In [4]:
df = getdata('ETHUSDT','2022-01-01')

In [5]:
class DataTrader(Strategy): 
    
    def init(self):
        close = self.data.Close 
        self.macd = self.I(ta.trend.macd, pd.Series(close))
        self.macd_signal = self.I(ta.trend.macd_signal, pd.Series(close))
        self.ema_100 = self.I(ta.trend.ema_indicator, pd.Series(close), window=100)
    
    def next(self):
        price = self.data.Close
        
        if crossover(self.macd, self.macd_signal) and price > self.ema_100: 
            sl = price*0.97 
            tp = price * 1.045
            self.buy(sl = sl , tp=tp )

In [6]:
bt = Backtest(df, DataTrader, cash=10000, commission= 0.0015 )

In [7]:
output=bt.run()

In [8]:
output

Start                     2022-01-01 00:00:00
End                       2022-10-23 16:00:00
Duration                    295 days 16:00:00
Exposure Time [%]                    5.577465
Equity Final [$]                 12022.497245
Equity Peak [$]                  12022.497245
Return [%]                          20.224972
Buy & Hold Return [%]              -64.622069
Return (Ann.) [%]                   25.499529
Volatility (Ann.) [%]               21.185499
Sharpe Ratio                         1.203631
Sortino Ratio                        2.678616
Calmar Ratio                         2.397463
Max. Drawdown [%]                  -10.636047
Avg. Drawdown [%]                   -2.547819
Max. Drawdown Duration      127 days 20:00:00
Avg. Drawdown Duration       14 days 19:00:00
# Trades                                   17
Win Rate [%]                        58.823529
Best Trade [%]                       4.344125
Worst Trade [%]                     -3.145867
Avg. Trade [%]                    

In [9]:
bt.plot()


Row(id='1506', ...)